In [2]:
%pip install requests_html;

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 15.0 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=8d4c616bac303bcda93872f5b8b32049b878b1290ce4bb39f1164867e1e49e7a
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [7]:
import requests
import urllib
import re
import time
import pandas as pd
from requests_html import HTML
from requests_html import HTMLSession

src_df = pd.read_csv('cnpj.csv')

def get_source(url):

    try:
        session = HTMLSession()
        response = session.get(url)
        return response

    except requests.exceptions.RequestException as e:
        print(e)

def scrape_google(query):

    query = urllib.parse.quote_plus(query)
    response = get_source("https://www.google.com.br/search?q=" + query)

    links = list(response.html.absolute_links)
    google_domains = ('https://www.google.',
                      'https://google.',
                      'https://webcache.googleusercontent.',
                      'http://webcache.googleusercontent.',
                      'https://policies.google.',
                      'https://support.google.',
                      'https://maps.google.')

    for url in links[:]:
        if url.startswith(google_domains):
            links.remove(url)

    return links

def get_results(query):

    query = urllib.parse.quote_plus(query)
    response = get_source("https://www.google.com.br/search?q=" + query)

    return response

def parse_results(response):

    css_identifier_result = ".tF2Cxc"
    css_identifier_title = "h3"
    css_identifier_link = ".yuRUbf a"
    css_identifier_text = ".VwiC3b"

    results = response.html.find(css_identifier_result)

    output = []

    for result in results:

        item = {
            'title': result.find(css_identifier_title, first=True).text,
            'link': result.find(css_identifier_link, first=True).attrs['href'],
            'text': result.find(css_identifier_text, first=True).text
        }

        output.append(item)

    return output

def google_search(query):
    response = get_results(query)
    return parse_results(response)

def get_cnpj(soup):

    phones = []
    phone = re.findall(r"([0-9]{2}[\.]?[0-9]{3}[\.]?[0-9]{3}[\/]?[0-9]{4}[-]?[0-9]{2})", soup)
    phones.extend(phone)

    if(len(phones) == 0): return ""
    else: return phones[0]

def get_phone(soup):

    phones = []
    phone = re.findall(r'\(\d{2}\)?\s\d{4,5}\-\d{4}', soup)
    phones.extend(phone)

    phone = re.findall(r'\(\b\(?\d{2}\)?\s\d{4,5}\-\d{4}', soup)
    phones.extend(phone)

    phone = re.findall(r'\(?\d{2}\)?\s\d{4,5}\-\d{4}', soup)
    phones.extend(phone)

    phone = re.findall(r'\(?\d{2}\)\d{4,5}\-\d{4}', soup)
    phones.extend(phone)

    if(len(phones) == 0): return ""
    else: return phones[0]

def formata(telefone: str) -> str:
    try:
        NUMEROS = "0123456789"
        numeros = ''
        for digito in telefone:
            if (digito in NUMEROS):
                numeros += digito

        mask = ""
        if (len(numeros) == 14):
            mask = "%s%s.%s%s%s.%s%s%s/%s%s%s%s-%s%s"

        mask = mask % tuple(numeros)
        return mask
    except:
        return ""


for i, row in src_df.iterrows():
    url = str(row['Dominios']) + "cnpj"
    try:
        results = str(google_search(url))
    except:
        print ('Unsucessful: ' + str(url))
        continue

    cnpj = get_cnpj(results)
    print(cnpj)


    src_df.loc[i,'CNPJ'] = cnpj
    time.sleep(0)

src_df["CNPJ"] = [formata(x) for x in src_df["CNPJ"]]
src_df.to_excel('output.xlsx', index=False)